# Install Packages

In [1]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

bash: line 6: fg: no job control


In [2]:
%%bash
bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
pip3 install /tmp/mecab-python-0.996

mecab-ko is already installed
mecab-ko-dic is already installed
mecab-python is already installed
Done.
Processing /tmp/mecab-python-0.996
  Created wheel for mecab-python: filename=mecab_python-0.996_ko_0.9.2-cp37-cp37m-linux_x86_64.whl size=141821 sha256=b6943d1c3c366d16ff5f92eb8485e3640c2c99cba67faa6be0962c6e7058c09d
  Stored in directory: /root/.cache/pip/wheels/40/7b/9f/2922869bef86c3354ae7034f7a3647c573ee1997c2dad0290a
Failed to build mecab-python
  Attempting uninstall: mecab-python
    Found existing installation: mecab-python 0.996-ko-0.9.2
    Uninstalling mecab-python-0.996-ko-0.9.2:
      Successfully uninstalled mecab-python-0.996-ko-0.9.2
    Running setup.py install for mecab-python: started
    Running setup.py install for mecab-python: finished with status 'done'


  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  DEPRECATION: mecab-python was installed using the legacy 'setup.py install' method, because a wheel could not be built for it. A possible replacement is to fix the wheel build issue reported above. You can find discussion regarding this at https://github.com/pypa/pip/issues/8368.


In [3]:
!pip install python-doctr
!pip install tensorflow-addons

# Set Environment

## Google Mount

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

## Load Packages

In [ ]:
import os
import string
import argparse

import cv2 as cv
import numpy as np
import pandas as pd

import torch
import torch.utils.data
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

from doctr.io import DocumentFile
from doctr.models import ocr_predictor

import OCR_modules.demo
from OCR_modules.model import Model
from OCR_modules.dataset import RawDataset, AlignCollate
from OCR_modules.utils import CTCLabelConverter, AttnLabelConverter

import konlpy
from konlpy.utils import pprint
from konlpy.tag import Mecab, Kkma, Komoran, Hannanum, Okt

from recommender_modules.etc_utils import *
from recommender_modules.ocr_utils import *
from recommender_modules.optimization import *
from recommender_modules.recommender_utils import *

from gensim.models import word2vec
from PIL import Image 


### CUDA Check 및 Mecab 불러오기
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
mecab = Mecab()
print(f'device: {device}')

# Load OCR Text Detection pretrained model

In [ ]:
# Load pretrained model
detection_model = ocr_predictor(pretrained=True)  

# Load dataset

In [4]:
# Load 영양성분 상하한 데이터 
nutrient_range_data = pd.read_excel('dataset/나이_연령별_영양성분.xlsx')

# Load 해먹남녀 Dataset
ingre_df = pd.read_csv('dataset/메뉴카테고리화_영양소_결합(ver3).csv',encoding = 'utf-8-sig')

# Set parameter

In [ ]:
# 전단지명 정의
image_filename = input('인식하고자 하는 전단지 파일명을 입력하세요 (.jpg는 제외): ')

# 사용자 정보 입력받기
user_sex = input('사용자의 성별을 입력하세요 (남자 또는 여자): ')
user_age = int(input('사용자의 연령대를 입력하세요 (6세 이상): '))
nutrient_boundary = get_nutrient_boundary(nutrient_range_data, user_sex, user_age)
print_user_nutrient_range(nutrient_boundary, user_sex, user_age)

# 사용자의 재료 입력받기
user_ingreds_list = [x.replace(" ",'') for x in input('현재 가지고 있는 식재료를 입력해주세요(입력 구분자: 콤마(", ")): ').split(", ")]

# 1. Text Detection & Text Recognition from Leaflet

In [ ]:
# Text Cropping결과를 저장할 폴더 생성
if 'contour_output' not in os.listdir():
    os.mkdir('contour_output')

if image_filename not in os.listdir('contour_output/'):
    os.mkdir(f'contour_output/{image_filename}')


# 고유 식재료들 파악
ingre_df['식재료들'] = ingre_df['변환된식재료'].apply(lambda x:x.split(','))
ingredient_list = []
for temp_list in ingre_df['식재료들'].tolist():
    for ingred in temp_list:
        ingredient_list.append(ingred)
ingredient_list = list(set(ingredient_list))


# OCR 결과 출력
ingredient_dictionary, ocr_words = get_OCR_result(leaflet_filename = image_filename, 
                                                  ingred_list = ingredient_list,
                                                  DETECTION_MODEL = detection_model,
                                                  MECAB = mecab)



print(f"\n전단지 인식 결과: {', '.join(ocr_words)}")

# 2. Association Rule Discovery Support-based Menu Recommendation System

In [ ]:
# 레시피별 식재료 파악
data_word = ingre_df['변환된식재료'].str.split(',')
recipe_list=[i for i in data_word]

# 사용자가 고려하고자 하는 최대 상위 최대 조합의 수 입력받기
user_num = int(input('상위 몇개의 조합을 확인하고 싶으신가요?: '))

# 조합에 따른 적절 레시피 추천
combinations_dict,topn_combinations = ingred_combination_asr(ingredient_dict = ingredient_dictionary, 
                                                             user_input = user_ingreds_list, 
                                                             ocr_output = ocr_words, 
                                                             recipes = recipe_list,
                                                             topn = user_num)

round(pd.DataFrame(combinations_dict, columns = ['식재료 조합', 'support']), 4)

In [ ]:
# 레시피에 해당하는 식재료들을 dict형태로 구분
df_recipe = ingre_df.loc[:,['레시피','식재료들']]
df_recipe['식재료'] = df_recipe['식재료들'].apply(lambda x: set(x))   

recipe_dictionary = {}
for i in range(len(df_recipe)):
    recipe_dictionary[df_recipe.loc[i,'레시피']] = set(df_recipe.loc[i,'식재료들'])


# 사용자가 고려하고자 하는 최대 상위 최대 조합의 수 입력받기
user_num = int(input('최대 몇개의 레시피를 고려하고 싶으신가요?: '))


# 레시피 추천
recommended_recipes = recommend_recipe_asr(recipe_dict = recipe_dictionary, 
                                           combinations_list = topn_combinations,
                                           topn = user_num)

recommended_recipes

# 3. Optimization

In [ ]:
df_optimize = ingre_df[ingre_df['레시피'].isin(recommended_recipes['레시피'].tolist())].reset_index(drop=True)

try: #### 최적화 시도
    solver1, optimal_key1, selected_df, optimize1_result = optimize1(df_optimize, nutrient_boundary)
    solver2, optimal_key2 = optimize2(optimize1_result)
    analysis_result(optimize1_result, selected_df, solver1, solver2, optimal_key1, optimal_key2, nutrient_boundary, user_ingreds_list, ocr_words) 

except: #### 최적화 불가 시 예외처리
    exception_log(df_optimize, user_ingreds_list, ocr_words)